In [ ]:
import numpy as np
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import zipfile

In [ ]:
if not os.path.exists('/kaggle/data/') :
    os.makedirs('/kaggle/data/')

In [ ]:
with zipfile.ZipFile('/kaggle/input/data-science-bowl-2018/stage1_train.zip', 'r') as existing_zip:
    existing_zip.extractall('/kaggle/data/train')

In [ ]:
with zipfile.ZipFile('/kaggle/input/data-science-bowl-2018/stage1_test.zip', 'r') as existing_zip:
    existing_zip.extractall('/kaggle/data/test')

In [ ]:
with zipfile.ZipFile('/kaggle/input/data-science-bowl-2018/stage1_solution.csv.zip', 'r') as existing_zip:
    existing_zip.extractall('/kaggle/data')

In [ ]:
with zipfile.ZipFile('/kaggle/input/data-science-bowl-2018/stage1_train_labels.csv.zip', 'r') as existing_zip:
    existing_zip.extractall('/kaggle/data')

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train_ids = next(os.walk('/kaggle/data/train'))[1]
test_ids = next(os.walk('/kaggle/data/test'))[1]

In [ ]:
test_ids

In [ ]:
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 3
from tqdm import tqdm

In [ ]:
from skimage.io import imread, imshow, imread_collection, concatenate_images

In [ ]:
TRAIN_PATH='/kaggle/data/train/'
TEST_PATH='/kaggle/data/test/'
img = imread(TRAIN_PATH+str(train_ids[1])+'/images/'+str(train_ids[1])+'.png')[:,:,:IMG_CHANNELS]

In [ ]:
imshow(img)

In [ ]:
img.shape

In [ ]:
img=[]
for i,id_ in enumerate(next(os.walk(TRAIN_PATH+str(train_ids[1])+'/masks/'))[2]):
    img.append(imread(TRAIN_PATH+str(train_ids[1])+'/masks/'+str(id_)))
img

In [ ]:
imshow(img[0])

In [ ]:
img[0].shape

In [ ]:
imshow(img[1])

In [ ]:
imshow(img[2])

In [ ]:
imshow(img[3])

In [ ]:
X_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
Y_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)

In [ ]:
from tqdm import tqdm

In [ ]:
from skimage.transform import resize

In [ ]:
for n,id_ in tqdm(enumerate(train_ids),total=len(train_ids)):
    img=imread(TRAIN_PATH+id_+'/images/'+id_+'.png')
    img=resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)[:,:,:IMG_CHANNELS]
    X_train[n]=img
    mask=np.zeros((IMG_HEIGHT,IMG_WIDTH,1),dtype=np.bool)
    for maskname in next(os.walk(TRAIN_PATH+id_+'/masks/'))[2]:
        mask_=imread(TRAIN_PATH+id_+'/masks/'+maskname)
        mask_=np.expand_dims(resize(mask_,(IMG_HEIGHT,IMG_WIDTH),mode='constant', preserve_range=True),axis=-1)
        mask=np.maximum(mask,mask_)
    Y_train[n] = mask

In [ ]:
img.shape

In [ ]:
X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
sizes_test = []

for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):
    img=imread(TEST_PATH +id_+ '/images/' + id_ + '.png')[:,:,:IMG_CHANNELS]
    sizes_test.append([img.shape[0], img.shape[1]])
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_test[n] = img


In [ ]:
import random
ix = random.randint(0, len(train_ids))

In [ ]:
imshow(X_train[ix])

In [ ]:
imshow(np.squeeze(Y_train[ix]))

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

In [ ]:
from keras import backend as K

In [ ]:
import keras

In [ ]:
keras.__version__

In [ ]:
def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.cast(y_pred > t, tf.int32)
        score, up_opt = tf.compat.v1.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)

In [ ]:
from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
s = Lambda(lambda x: x / 255) (inputs)

c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (s)
c1 = Dropout(0.1) (c1)
c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c1)
p1 = MaxPooling2D((2, 2)) (c1)

c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p1)
c2 = Dropout(0.1) (c2)
c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c2)
p2 = MaxPooling2D((2, 2)) (c2)

c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p2)
c3 = Dropout(0.2) (c3)
c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c3)
p3 = MaxPooling2D((2, 2)) (c3)

c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p3)
c4 = Dropout(0.2) (c4)
c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c4)
p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p4)
c5 = Dropout(0.3) (c5)
c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c5)

u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c5)
u6 = concatenate([u6, c4])
c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u6)
c6 = Dropout(0.2) (c6)
c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c6)

u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
u7 = concatenate([u7, c3])
c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u7)
c7 = Dropout(0.2) (c7)
c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c7)

u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c7)
u8 = concatenate([u8, c2])
c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u8)
c8 = Dropout(0.1) (c8)
c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c8)

u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c8)
u9 = concatenate([u9, c1], axis=3)
c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u9)
c9 = Dropout(0.1) (c9)
c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c9)

outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)

model = Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
earlystopper = EarlyStopping(patience=5, verbose=1)
checkpointer = ModelCheckpoint('model-dsbowl2018.h5', verbose=1, save_best_only=True)

results = model.fit(X_train, Y_train, validation_split=0.1, batch_size=16, epochs=50, 
                    callbacks=[earlystopper, checkpointer])

In [ ]:
model = load_model('model-dsbowl2018.h5', custom_objects={'mean_iou': mean_iou})
preds_test = model.predict(X_test, verbose=1)

preds_test_upsampled = []
for i in range(len(preds_test)):
    preds_test_upsampled.append(resize(np.squeeze(preds_test[i]), 
                                       (sizes_test[i][0], sizes_test[i][1]), 
                                       mode='constant', preserve_range=True))

In [ ]:
from skimage.morphology import label

In [ ]:
def rle_encoding(x):
    dots = np.where(x.T.flatten() == 1)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if (b>prev+1): run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths

def prob_to_rles(x, cutoff=0.5):
    lab_img = label(x > cutoff)
    for i in range(1, lab_img.max() + 1):
        yield rle_encoding(lab_img == i)

In [ ]:
new_test_ids = []
rles = []
for n, id_ in enumerate(test_ids):
    rle = list(prob_to_rles(preds_test_upsampled[n]))
    rles.extend(rle)
    new_test_ids.extend([id_] * len(rle))

In [ ]:
sub = pd.DataFrame()
sub['ImageId'] = new_test_ids
sub['EncodedPixels'] = pd.Series(rles).apply(lambda x: ' '.join(str(y) for y in x))
sub.to_csv('sub-dsbowl2018.csv', index=False)